# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 952.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [2]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [4]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.5-flash-preview"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [5]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745045941-WeogRiSW0RgZ9MXYgQzh', 'provider': 'Google', 'model': 'google/gemini-2.5-flash-preview', 'object': 'chat.completion', 'created': 1745045941, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'STOP', 'index': 0, 'message': {'role': 'assistant', 'content': 'That\'s the million-dollar, or perhaps the eternal, question! The meaning of life is one of the most profound and debated topics throughout human history, and there\'s no single, universally accepted answer. Here\'s a breakdown of why it\'s so complex and some common perspectives:\n\n**Why is there no single answer?**\n\n* **Subjectivity:** What gives life meaning is deeply personal. What is meaningful to one person might be completely irrelevant to another.\n* **Lack of Scientific Proof:** Meaning isn\'t a physical entity that can be measured or observed scientifically.\n* **Cultural and Philosophical Diversity:** Different cultures, religions, and philosophies offer vastly differe

In [6]:
data = response.json()
data['choices'][0]['message']['content']


'That\'s the million-dollar, or perhaps the eternal, question! The meaning of life is one of the most profound and debated topics throughout human history, and there\'s no single, universally accepted answer. Here\'s a breakdown of why it\'s so complex and some common perspectives:\n\n**Why is there no single answer?**\n\n* **Subjectivity:** What gives life meaning is deeply personal. What is meaningful to one person might be completely irrelevant to another.\n* **Lack of Scientific Proof:** Meaning isn\'t a physical entity that can be measured or observed scientifically.\n* **Cultural and Philosophical Diversity:** Different cultures, religions, and philosophies offer vastly different interpretations.\n* **Evolution of Understanding:** Our understanding of meaning often changes throughout our lives as we grow, learn, and experience new things.\n\n**Common Perspectives on the Meaning of Life:**\n\nHere are some common ideas and frameworks for understanding the meaning of life:\n\n**1. 

## 2. Using OpenAI Agents SDK

In [7]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

Function calls itself,
Breaks problem to small pieces,
Base case stops the loop.


# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](https://github.com/panaversity/learn-agentic-ai/blob/main/01_ai_agents_first/02_openrouter/openrouter.png?raw=1)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```